<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# IA Framework.
## Lab 1  - Introduction to Pyspark.
#### Part 3 Introduction à la librairie  [SparkML](https://spark.apache.org/docs/latest/ml-guide.html) (ou *MLlib  DataFrame-based API*) de  <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> 

# 

**Résumé**: Ce tutoriel continue l'initiation à [Spark](https://spark.apache.org/) à l'aide de commandes en Python utilisant l'API  [`PySpark`](http://spark.apache.org/docs/latest/api/python/). CE calepin mainipule des *DataFrames*

## Introduction
Depuis Spark 2.0 la librairie MLlib, qui manipule éxclusivement des RDDs est en maintenance. Elle peut toujours être utilisée, mais il n'y aura pas de nouveaux outils développé pour cette librairie.

La principale librairie de *machine learning* pour Spark est maintenant SparkML. SparkML manipule exclusivement des *DataFrame*, plus faciles d'utilisation que les RDD et permet l'utilisation d'autres services Spark tels que *Spark Datasources*, *SQL queries*...

Si MLlib est aujourd'hui plus complète, SparkML devrait posséder exactement les même fonctions que MLlib dans la version 2.3 de Spark (Aujourd'hui la version 2.2 est disponible). La librairie MLlib, qui manipule exclusivement des RDD n'existera plus dans Spark 3.0


SparkML n'est pas le nom définitif que portera la librairie de machine learning de spark. C'est un nom temporaire créer pour déveloper un outil basé sur les DataFrame. MLlib sera le nom définitif de cette librairie.

##  Statistique élémentaire avec <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> et  [SparkML](https://spark.apache.org/docs/latest/ml-guide.html)

La plupart des fonctions de statistique élémentaire de *MLlib*, décrites dans le calepin 2 de cette introduction à pyspark ne sont pas encore disponibles dans la librairie *SparkML*. Seule la fonction de correlation et le test d'hypothèse du χ² sont disponible. 

In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Correlation

La fonction `pyspark.ml.stat.Correlation` permet de calculer les correlations entre les colonnes d'un *DataFrame*. Les correlations disponible sont celles de *Pearson* et *Spearman*. 

In [ ]:
from numpy import array
from pyspark.ml.linalg import Vectors
# vecteur "dense"
# à partir de numpy
denseVec1=array([1.0,0.0,2.0,4.0,0.0])
# en utilisant la classe Vectors
denseVec2=Vectors.dense([1.0,0.0,2.0,4.0,0.0])

In [ ]:
from numpy import array
from pyspark.ml.linalg import Vectors
# vecteur "dense"
# à partir de numpy
denseVec1=array([1.0,0.0,2.0,4.0,0.0])
# en utilisant la classe Vectors
denseVec2=Vectors.dense([1.0,0.0,2.0,4.0,0.0])

In [ ]:
sparseVec1 = Vectors.sparse(5, {0: 1.0, 2: 2.0, 3: 4.0})
sparseVec2 = Vectors.sparse(5, [0, 2, 3], [1.0, 2.0, 4.0])

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]


df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

### Summary Statistics

In [ ]:
r2

### Test d'Hypothèse

Spark.ml supporte actuellement le test du Chi-2 de Pearson. Ce test permet d'effectuer un test d'indépendance pour chaque *features* vis à vis du *label*. 

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

## ML Pipeline

La librairie *SparkML*, contrairement à *MLlib*, est basé sur la notion de **ML Pipeline**. 

Un **ML Pipeline** permet de combiner différentes étapes de traitement, allant du nettoyage des données jusqu'a l'étape d'apprentissage en un seul objet appelé *pipeline* ou *workflow*. 

### Estimator, Transformer, and Param

La construction d'un **ML Pipeline** est effectué à partir de trois type d'objets décrits ci-dessous :


 * **Transformer**: C'est un algorithme qui permet de transformer un *DataFrame* en un autre *DataFrame*. Dans la pluspart des cas, le nouveau *DataFrame* est identique au premier avec une colonne supplémentaire. Exemple de **Transformer**: 
     * Un modèle d'apprentissage va prendre en entrée un DataFrame de variables et retourner un nouveau DataFrame avec les variables et une nouvelle colonne correspondant à la prédiction.
     * Le Transformer *StringIndexer* va prendre en entrée un DataFrame possédant une colonne de texte et retourner un DataFrame avec la même colonne texte et une nouvelle colonne ou les textes sont remplacés par une valeur numérique.
 
* **Estimator**: C'est un algorithme qui peut-être appliqué sur un DataFrame afin de produire un **Transformer**. Exemple d'**Estimator**:
    * Un algorithme d'apprentissage est un **Estimator**. Une fois appliqué sur un DataFrame, il va produire un modèle d'apprentissage qui sera un **Transformer**, comme décrit précédemment.

 * **Parameter**: Chaque **Transformer** et **Estimators** partage une même API pour spécifier leurs paramètres.

#### Exemple : Regression Logistique

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# DataFrame D'apprentissage
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])


# DataFrame de Test
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

Création d'un objet `LogisticRegression` en spécifiant ses paramètres. L'objet `lr` créé est un `Estimator``


In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability')
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

Appliquer l'Estimator sur le DataFrame d'apprentissage. 
L'objet qui en résulte, le modèle d'apprentissage, est un `Transformer`.

In [ ]:
model = lr.fit(training)

Appliquer le `Transformer` sur le `DataFrame` de Test 'test'.

In [ ]:
prediction = model.transform(test)

Il en resulte un nouveauDataFrame `prediction` qui correspond au DataFrame 'result' auquel 
on été ajoutés une colonne `prediction` et une colonne `probability`.
Les noms de ces nouvelles colonnes ont été spécifiés dans les paramètres lors de l'instanciation 
de l'Estimator `lr` puis transmis au Transformer `model``
.

In [ ]:
result = prediction.select("features", "label", "probability", "prediction") \
    .collect()

Affichage des résultats.

In [ ]:
for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.probability, row.prediction))

### Pipeline

Un **Pipeline** est un enchainement de plusieurs **Transformers** et **Estimators** afin de spécifier un processus entier de Machine Learning. Par exemple, pour effectuer de l'apprentissage statistique sur des données textuelles, ces différentes étapes sont appliquées les unes à la suite des autres: 

 * Découpage du texte en liste de mot
 * Conversion en variable numérique
 * Aprentissahe sur les données numérique
 * Prédiction 

Toutes ces étapes peuvent être résumé dans un seul objet appelé **Pipeline**.

#### Exemple : Tokenize, Hash et Regression logistique

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# DataFrame d'Apprentissage
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])


# DataFrame Test.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])


On Configure un pipeline qui consiste en 3 étapes :
 
 1/ *Tokenizer*. Spécifier la colonne d'entrée 'text' et la colonne de sortie 'words'.

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

2/ *Hash*. La colonne d'entrée est spécifiée ici comme étant la colonne de sortie de l'étape précédente.


In [ ]:
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

3/ Regression Logistique

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.001)

Configuration du Pipeline comme la succession des étapes précédentes.

In [ ]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])


Appliquer toutes les étapes sur le DataFrame d'apprentissage.

In [ ]:
model = pipeline.fit(training)

Prediction

In [ ]:
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))